In [1]:
import bigframes.pandas as bpd
import bigframes.ml.llm as llm
import numpy as np
import pandas as pd

In [2]:
model = llm.TextEmbeddingGenerator(model_name='text-embedding-005', connection_name='projects/financial-fraud-detection-1/locations/us/connections/vertex_AI_connection')

/usr/local/lib/python3.11/dist-packages/bigframes/core/global_session.py:103: DefaultLocationWarning: No explicit location is set, so using location US for the session.
  _global_session = bigframes.session.connect(


CREATE OR REPLACE MODEL `financial-fraud-detection-1`.`_ece87972ce3ce2b55ab9b47413e61ef5f70dbc1f`.`1abaa11b2a12496ba3d6a74d408d4feb`
REMOTE WITH CONNECTION `financial-fraud-detection-1.us.vertex_AI_connection`
OPTIONS(
  endpoint='text-embedding-005')

In [3]:
model.get_params()

{'connection_name': 'financial-fraud-detection-1.us.vertex_AI_connection',
 'model_name': 'text-embedding-005',
 'session': <bigframes.session.Session at 0x7be51a1ad850>}

In [5]:
df_edgar = bpd.read_gbq("financial-fraud-detection-1.EDGAR.edgar_corpus_sample_small")

In [6]:
df_edgar

,filename,cik,year,section_1,section_1A,section_1B,section_2,section_3,section_4,section_5,...,section_8,section_9,section_9A,section_9B,section_10,section_11,section_12,section_13,section_14,section_15
0,50104_1993.txt,50104,1993,"ITEM 1. BUSINESS Tesoro Petroleum Corporation,...",,,ITEM 2. PROPERTIES See information appearing u...,ITEM 3. LEGAL PROCEEDINGS (a) The Company is s...,ITEM 4. SUBMISSION OF MATTERS TO A VOTE OF SEC...,ITEM 5. MARKET FOR THE REGISTRANT'S COMMON EQU...,...,ITEM 8. FINANCIAL STATEMENTS AND SUPPLEMENTARY...,ITEM 9. CHANGES IN AND DISAGREEMENTS WITH ACCO...,,,ITEM 10. DIRECTORS AND EXECUTIVE OFFICERS OF T...,ITEM 11. EXECUTIVE COMPENSATION Information re...,ITEM 12. SECURITY OWNERSHIP OF CERTAIN BENEFIC...,ITEM 13. CERTAIN RELATIONSHIPS AND RELATED TRA...,"ITEM 14. EXHIBITS, FINANCIAL STATEMENT SCHEDUL...",
1,315189_1993.txt,315189,1993,ITEM 1. BUSINESS. PRODUCTS Deere & Company (Co...,,,"ITEM 2. PROPERTIES. See ""Manufacturing"" in Ite...",ITEM 3. LEGAL PROCEEDINGS. The Company is subj...,ITEM 4. SUBMISSION OF MATTERS TO A VOTE OF SEC...,ITEM 5. MARKET FOR REGISTRANT'S COMMON EQUITY ...,...,ITEM 8. FINANCIAL STATEMENTS AND SUPPLEMENTARY...,ITEM 9. CHANGES IN AND DISAGREEMENTS WITH ACCO...,,,ITEM 10. DIRECTORS AND EXECUTIVE OFFICERS OF T...,ITEM 11. EXECUTIVE COMPENSATION. The informati...,ITEM 12. SECURITY OWNERSHIP OF CERTAIN BENEFIC...,ITEM 13. CERTAIN RELATIONSHIPS AND RELATED TRA...,"ITEM 14. EXHIBITS, FINANCIAL STATEMENT SCHEDUL...",
2,1800_1993.txt,1800,1993,ITEM 1. BUSINESS GENERAL DEVELOPMENT OF BUSINE...,,,ITEM 2. PROPERTIES The Company's corporate off...,ITEM 3. LEGAL PROCEEDINGS The Company is invol...,ITEM 4. SUBMISSION OF MATTERS TO A VOTE OF SEC...,ITEM 5. MARKET FOR THE REGISTRANT'S COMMON EQU...,...,ITEM 8. FINANCIAL STATEMENTS AND SUPPLEMENTARY...,ITEM 9. DISAGREEMENTS ON ACCOUNTING AND FINANC...,,,ITEM 10. DIRECTORS AND EXECUTIVE OFFICERS OF T...,ITEM 11. EXECUTIVE COMPENSATION The material i...,ITEM 12. SECURITY OWNERSHIP OF CERTAIN BENEFIC...,ITEM 13. CERTAIN RELATIONSHIPS AND RELATED TRA...,"ITEM 14. EXHIBITS, FINANCIAL STATEMENT SCHEDUL...",
3,1137091_2020.htm,1137091,2020,Item 1. Business. General Business Overview Po...,Item 1A. Risk Factors. The Company’s business ...,Item 1B. Unresolved Staff Comments. None. Item 2.,Item 2. Properties. The Company’s operations a...,Item 3. Legal Proceedings. See Note 10. Commit...,Item 4. Mine Safety Disclosures. Not applicabl...,"Item 5. Market for Registrant’s Common Equity,...",...,Item 8. Financial Statements and Supplementary...,Item 9. Changes in and Disagreements with Acco...,Item 9A. Controls and Procedures. Evaluation o...,Item 9B. Other Information. None. PART III Ite...,"Item 10. Directors, Executive Officers and Cor...",Item 11. Executive Compensation. The informati...,Item 12. Security Ownership of Certain Benefic...,Item 13. Certain Relationships and Related Tra...,Item 14. Principal Accounting Fees and Service...,"Item 15. Exhibits, Financial Statement Schedul..."
4,1067983_2020.htm,1067983,2020,Item 1. Business Description Berkshire Hathawa...,Item 1A. Risk Factors Berkshire and its subsid...,Item 1B. Unresolved Staff Comments None. Item 2.,Item 2. Description of Properties The properti...,Item 3. Legal Proceedings Berkshire and its su...,Item 4. Mine Safety Disclosures Information re...,"Item 5. Market for Registrant’s Common Equity,...",...,Item 8. Financial Statements and Supplementary...,Item 9. Changes in and Disagreements with Acco...,Item 9A. Controls and Procedures At the end of...,Item 9B. Other Information None Part III Excep...,,,,,,Item 15. Exhibits and Financial Statement Sche...


In [8]:
df_edgar.info()

<class 'bigframes.dataframe.DataFrame'>
Index: 5 entries, 0 to 4
Data columns (total 23 columns):
  #  Column      Non-Null Count    Dtype
---  ----------  ----------------  -------
  0  filename    5 non-null        string
  1  cik         5 non-null        Int64
  2  year        5 non-null        Int64
  3  section_1   5 non-null        string
  4  section_1A  5 non-null        string
  5  section_1B  5 non-null        string
  6  section_2   5 non-null        string
  7  section_3   5 non-null        string
  8  section_4   5 non-null        string
  9  section_5   5 non-null        string
 10  section_6   5 non-null        string
 11  section_7   5 non-null        string
 12  section_7A  5 non-null        string
 13  section_8   5 non-null        string
 14  section_9   5 non-null        string
 15  section_9A  5 non-null        string
 16  section_9B  5 non-null        string
 17  section_10  5 non-null        string
 18  section_11  5 non-null        string
 19  section_12  5 non

In [9]:
# averaged_embedding_pd = pd.DataFrame(a.to_pandas().to_list()).mean(axis = 0)

In [10]:
# # A remote function that takes a single string, chunks it, and returns an averaged embedding.
# @bpd.remote_function(
#     cloud_function_service_account="default",
#     output_type=list[float]
# )
def chunk_and_embed_text(text: str) -> list[float]:
    """Chunks a long text and returns an averaged embedding."""

    if not text:
      return [0.0] * 768
    if not isinstance(text, str):
        return [0.0] * 768
    if not str(text).strip():
        return [0.0] * 768

    chunk_size = 1000
    words = text.split()
    chunks = [' '.join(words[i:i + chunk_size]) for i in range(0, len(words), chunk_size)]

    chunks_series = pd.Series(chunks)
    chunks_series.dropna(inplace=True)
    if chunks_series.empty:
        return [0.0] * 768
    embeddings_df = model.predict(chunks_series)

    if not embeddings_df.empty:
        # Flatten the vectors into a single Series to apply the mean
        # This is the correct way to perform the element-wise mean in BigFrames
        vector_series = embeddings_df['ml_generate_embedding_result']
        vector_df = pd.DataFrame(vector_series.to_pandas().to_list())

        # Calculate the mean of the DataFrame's columns, which results in a Series
        averaged_embedding_series = vector_df.mean(axis = 0)

        # Convert the resulting BigFrames Series back to a Python list
        print("One cell completed with embeddings")
        return averaged_embedding_series.to_list()

    # Return an empty list if no embeddings are generated
    return [0.0] * 768

In [11]:
# @bpd.remote_function(
#     cloud_function_service_account="default",
#     output_type=list[float]
# )
# def chunk_and_embed_text_func(text: str) -> list[float]:
#     """This function just points to your remote implementation."""
#     return chunk_and_embed_text(text)

In [12]:
len(df_edgar.columns)

23

In [13]:
column_names_to_be_embedded = set(df_edgar.columns) - set(['filename', 'cik', 'year'])

In [14]:
df_embedding = df_edgar[['filename', 'cik', 'year']]

In [15]:
type(df_embedding)

bigframes.dataframe.DataFrame

In [16]:
df_edgar_pd = df_edgar.to_pandas()

In [17]:
for col in column_names_to_be_embedded:
  print(col)

section_7
section_9
section_12
section_10
section_4
section_8
section_14
section_1A
section_11
section_15
section_5
section_13
section_7A
section_1B
section_9B
section_2
section_6
section_3
section_9A
section_1


In [18]:
df_embedding.info()

<class 'bigframes.dataframe.DataFrame'>
Index: 5 entries, 0 to 4
Data columns (total 3 columns):
  #  Column    Non-Null Count    Dtype
---  --------  ----------------  -------
  0  filename  5 non-null        string
  1  cik       5 non-null        Int64
  2  year      5 non-null        Int64
dtypes: Int64(2), string(1)
memory usage: 160 bytes


In [19]:
for col in column_names_to_be_embedded:
  new_col_name = col + 'embedding'
  df_embedding[new_col_name] = [chunk_and_embed_text(i) for i in df_edgar_pd[col]]
  print("-----------------------------------------------------\n Column that is completed" + col)

/usr/local/lib/python3.11/dist-packages/bigframes/core/array_value.py:108: PreviewWarning: JSON column interpretation as a custom PyArrow extention in
`db_dtypes` is a preview feature and subject to change.
  warnings.warn(msg, bfe.PreviewWarning)


One cell completed with embeddings


/usr/local/lib/python3.11/dist-packages/bigframes/core/array_value.py:108: PreviewWarning: JSON column interpretation as a custom PyArrow extention in
`db_dtypes` is a preview feature and subject to change.
  warnings.warn(msg, bfe.PreviewWarning)


One cell completed with embeddings


/usr/local/lib/python3.11/dist-packages/bigframes/core/array_value.py:108: PreviewWarning: JSON column interpretation as a custom PyArrow extention in
`db_dtypes` is a preview feature and subject to change.
  warnings.warn(msg, bfe.PreviewWarning)


...

/usr/local/lib/python3.11/dist-packages/bigframes/core/array_value.py:108: PreviewWarning: JSON column interpretation as a custom PyArrow extention in
`db_dtypes` is a preview feature and subject to change.
  warnings.warn(msg, bfe.PreviewWarning)


One cell completed with embeddings


/usr/local/lib/python3.11/dist-packages/bigframes/core/array_value.py:108: PreviewWarning: JSON column interpretation as a custom PyArrow extention in
`db_dtypes` is a preview feature and subject to change.
  warnings.warn(msg, bfe.PreviewWarning)


One cell completed with embeddings


-----------------------------------------------------
 Column that is completedsection_1


In [20]:
df_embedding

,filename,cik,year,section_7embedding,section_9embedding,section_12embedding,section_10embedding,section_4embedding,section_8embedding,section_14embedding,...,section_5embedding,section_13embedding,section_7Aembedding,section_1Bembedding,section_9Bembedding,section_2embedding,section_6embedding,section_3embedding,section_9Aembedding,section_1embedding
0,50104_1993.txt,50104,1993,[-3.47243063e-02 1.94188505e-02 -8.92389864e-...,[-1.55314188e-02 -2.45964527e-03 -3.88711654e-...,[-3.79250757e-02 -4.47099060e-02 -3.67634818e-...,[-4.57181484e-02 -5.33069260e-02 -1.22106224e-...,[-0.05072239 -0.01314292 -0.03440231 -0.023048...,[-3.04003223e-02 4.79081871e-03 -5.41995295e-...,[ 3.38502717e-03 -5.04764915e-03 -5.59636764e-...,...,[-6.61806986e-02 -2.95599401e-02 -3.89909521e-...,[-4.50329408e-02 -7.63122365e-02 -3.43960524e-...,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,[-1.83147509e-02 -4.80058193e-02 -3.25606801e-...,[ 1.19868275e-02 -3.50940004e-02 -3.42585109e-...,[-5.95068801e-02 2.10417886e-03 -2.92739635e-...,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,[-0.03292404 0.00087296 -0.04182 -0.004596...
1,315189_1993.txt,315189,1993,[-1.35571547e-02 -1.56442206e-02 -2.92028040e-...,[-1.83812398e-02 1.81908242e-03 -2.49630474e-...,[-3.85654159e-02 -1.78559888e-02 -4.98842299e-...,[-5.09253442e-02 -3.28539982e-02 -1.21700037e-...,[-4.47579399e-02 -1.16266981e-02 -4.29858454e-...,[-9.26065768e-05 -5.11534102e-02 -4.29588519e-...,[ 2.28945124e-02 5.21549085e-03 -1.74354800e-...,...,[-4.36332561e-02 -1.45736393e-02 -3.86644038e-...,[-5.07865660e-02 -4.80615199e-02 -3.68813202e-...,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,[ 1.60061345e-02 2.85344739e-02 2.26290263e-...,[-1.48224207e-02 -4.50210385e-02 -6.59380779e-...,[-1.44546749e-02 -2.31604483e-02 -1.49410742e-...,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,[ 2.29252045e-02 2.62432406e-03 -4.36920724e-...
2,1800_1993.txt,1800,1993,[-0.01756296 -0.01733708 -0.01346771 -0.013578...,[-1.56782791e-02 -6.42858818e-03 -3.48928086e-...,[-4.16508093e-02 -2.62772869e-02 -3.11164446e-...,[-6.22933321e-02 -4.06846032e-02 -2.88069844e-...,[-0.01235421 -0.05187324 -0.01839188 0.048709...,[-0.01758242 -0.04470409 -0.02377128 -0.002298...,[ 3.80702619e-03 -6.57393262e-02 -4.26834710e-...,...,[-3.55412625e-02 -8.17985367e-03 -4.37800437e-...,[-4.13229093e-02 -6.52043819e-02 -4.08298895e-...,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,[-1.38139455e-02 -9.38619545e-04 -1.91317741e-...,[-2.47520432e-02 -3.38479057e-02 -2.15717610e-...,[-0.05118279 -0.03660664 -0.03556744 -0.023323...,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,[-8.46420446e-03 -1.74391275e-02 -3.04021587e-...
3,1137091_2020.htm,1137091,2020,[-2.10153785e-02 2.59747695e-03 -3.92209039e-...,[-4.25587595e-02 -2.09636036e-02 -6.31605536e-...,[-6.73391223e-02 -1.49382157e-02 -3.27744223e-...,[-3.24487165e-02 -1.60200917e-03 -5.89513499e-...,[-3.31896767e-02 1.45743862e-02 -3.72849405e-...,[-1.28673098e-02 -8.76872710e-03 -2.86414517e-...,[-7.22742230e-02 -2.71525867e-02 -2.11981349e-...,...,[-2.52595507e-02 -1.44289935e-03 -3.42903212e-...,[-0.05171408 -0.06189608 -0.03306692 0.002361...,[-4.44080308e-02 3.40547636e-02 -2.83501763e-...,[-6.44871220e-02 -3.82172340e-03 -1.57967471e-...,[-5.18303849e-02 -1.67017803e-02 -4.17982489e-...,[-9.54162516e-03 -8.53198580e-03 7.73510430e-...,[-0.02995299 -0.00425722 -0.03677021 0.041000...,[-5.38757481e-02 -4.11248840e-02 -2.54752636e-...,[-3.16515304e-02 -1.75493170e-02 -9.58232504e-...,[-5.22894418e-03 -6.85504824e-03 -1.68921131e-...
4,1067983_2020.htm,1067983,2020,[-3.98591966e-04 -6.93545857e-03 -3.90583078e-...,[-3.78713012e-02 -2.25618221e-02 -6.61137030e-...,[

In [21]:
df_embedding.info()

<class 'bigframes.dataframe.DataFrame'>
Index: 5 entries, 0 to 4
Data columns (total 23 columns):
  #  Column               Non-Null Count    Dtype
---  -------------------  ----------------  ---------------------------
  0  filename             5 non-null        string
  1  cik                  5 non-null        Int64
  2  year                 5 non-null        Int64
  3  section_7embedding   5 non-null        list<item: double>[pyarrow]
  4  section_9embedding   5 non-null        list<item: double>[pyarrow]
  5  section_12embedding  5 non-null        list<item: double>[pyarrow]
  6  section_10embedding  5 non-null        list<item: double>[pyarrow]
  7  section_4embedding   5 non-null        list<item: double>[pyarrow]
  8  section_8embedding   5 non-null        list<item: double>[pyarrow]
  9  section_14embedding  5 non-null        list<item: double>[pyarrow]
 10  section_1Aembedding  5 non-null        list<item: double>[pyarrow]
 11  section_11embedding  5 non-null        list<item

In [27]:
df_embedding.to_pandas().iloc[:, 3:]

,section_7embedding,section_9embedding,section_12embedding,section_10embedding,section_4embedding,section_8embedding,section_14embedding,section_1Aembedding,section_11embedding,section_15embedding,section_5embedding,section_13embedding,section_7Aembedding,section_1Bembedding,section_9Bembedding,section_2embedding,section_6embedding,section_3embedding,section_9Aembedding,section_1embedding
0,[-3.47243063e-02 1.94188505e-02 -8.92389864e-...,[-1.55314188e-02 -2.45964527e-03 -3.88711654e-...,[-3.79250757e-02 -4.47099060e-02 -3.67634818e-...,[-4.57181484e-02 -5.33069260e-02 -1.22106224e-...,[-0.05072239 -0.01314292 -0.03440231 -0.023048...,[-3.04003223e-02 4.79081871e-03 -5.41995295e-...,[ 3.38502717e-03 -5.04764915e-03 -5.59636764e-...,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,[-4.79705371e-02 -6.59401938e-02 -2.51594633e-...,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,[-6.61806986e-02 -2.95599401e-02 -3.89909521e-...,[-4.50329408e-02 -7.63122365e-02 -3.43960524e-...,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,[-1.83147509e-02 -4.80058193e-02 -3.25606801e-...,[ 1.19868275e-02 -3.50940004e-02 -3.42585109e-...,[-5.95068801e-02 2.10417886e-03 -2.92739635e-...,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,[-0.03292404 0.00087296 -0.04182 -0.004596...
1,[-1.35571547e-02 -1.56442206e-02 -2.92028040e-...,[-1.83812398e-02 1.81908242e-03 -2.49630474e-...,[-3.85654159e-02 -1.78559888e-02 -4.98842299e-...,[-5.09253442e-02 -3.28539982e-02 -1.21700037e-...,[-4.47579399e-02 -1.16266981e-02 -4.29858454e-...,[-9.26065768e-05 -5.11534102e-02 -4.29588519e-...,[ 2.28945124e-02 5.21549085e-03 -1.74354800e-...,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,[-7.09736049e-02 -2.02373546e-02 -4.63324003e-...,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,[-4.36332561e-02 -1.45736393e-02 -3.86644038e-...,[-5.07865660e-02 -4.80615199e-02 -3.68813202e-...,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,[ 1.60061345e-02 2.85344739e-02 2.26290263e-...,[-1.48224207e-02 -4.50210385e-02 -6.59380779e-...,[-1.44546749e-02 -2.31604483e-02 -1.49410742e-...,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,[ 2.29252045e-02 2.62432406e-03 -4.36920724e-...
2,[-0.01756296 -0.01733708 -0.01346771 -0.013578...,[-1.56782791e-02 -6.42858818e-03 -3.48928086e-...,[-4.16508093e-02 -2.62772869e-02 -3.11164446e-...,[-6.22933321e-02 -4.06846032e-02 -2.88069844e-...,[-0.01235421 -0.05187324 -0.01839188 0.048709...,[-0.01758242 -0.04470409 -0.02377128 -0.002298...,[ 3.80702619e-03 -6.57393262e-02 -4.26834710e-...,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,[-6.68694451e-02 -4.11794297e-02 -2.24579200e-...,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,[-3.55412625e-02 -8.17985367e-03 -4.37800437e-...,[-4.13229093e-02 -6.52043819e-02 -4.08298895e-...,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,[-1.38139455e-02 -9.38619545e-04 -1.91317741e-...,[-2.47520432e-02 -3.38479057e-02 -2.15717610e-...,[-0.05118279 -0.03660664 -0.03556744 -0.023323...,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,[-8.46420446e-03 -1.74391275e-02 -3.04021587e-...
3,[-2.10153785e-02 2.59747695e-03 -3.92209039e-...,[-4.25587595e-02 -2.09636036e-02 -6.31605536e-...,[-6.73391223e-02 -1.49382157e-02 -3.27744223e-...,[-3.24487165e-02 -1.60200917e-03 -5.89513499e-...,[-3.31896767e-02 1.45743862e-02 -3.72849405e-...,[-1.28673098e-02 -8.76872710e-03 -2.86414517e-...,[-7.22742230e-02 -2.71525867e-02 -2.11981349e-...,[-2.56591383e-02 4.31199711e-03 -2.39572942e-...,[-6.45152405e-02 -2.94506457e-02 -1.84390489e-...,[-5.07084047e-03 -2.76713558e-02 -2.69693565e-...,[-2.52595507e-02 -1.44289935e-03 -3.42903212e-...,[-0.05171408 -0.06189608 -0.03306692 0.002361...,[-4.44080308e-02 

In [29]:
df_embedding.to_pandas().iloc[:, 3:].applymap(len)

/tmp/ipython-input-2795087265.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_embedding.to_pandas().iloc[:, 3:].applymap(len)


,section_7embedding,section_9embedding,section_12embedding,section_10embedding,section_4embedding,section_8embedding,section_14embedding,section_1Aembedding,section_11embedding,section_15embedding,section_5embedding,section_13embedding,section_7Aembedding,section_1Bembedding,section_9Bembedding,section_2embedding,section_6embedding,section_3embedding,section_9Aembedding,section_1embedding
0,768,768,768,768,768,768,768,768,768,768,768,768,768,768,768,768,768,768,768,768
1,768,768,768,768,768,768,768,768,768,768,768,768,768,768,768,768,768,768,768,768
2,768,768,768,768,768,768,768,768,768,768,768,768,768,768,768,768,768,768,768,768
3,768,768,768,768,768,768,768,768,768,768,768,768,768,768,768,768,768,768,768,768
4,768,768,768,768,768,768,768,768,768,768,768,768,768,768,768,768,768,768,768,768


In [30]:
df_embedding.to_gbq("financial-fraud-detection-1.EDGAR.edgar_corpus_small_embeddings", if_exists='replace')

'financial-fraud-detection-1.EDGAR.edgar_corpus_small_embeddings'

In [37]:
type(df_embedding.to_pandas().iloc[:, 3:])

pandas.core.frame.DataFrame

In [42]:
df_embedding_only = df_embedding.to_pandas().iloc[:, 3:]

In [44]:
df_embedding_only.iloc[0]

,0
section_7embedding,[-3.47243063e-02 1.94188505e-02 -8.92389864e-...
section_9embedding,[-1.55314188e-02 -2.45964527e-03 -3.88711654e-...
section_12embedding,[-3.79250757e-02 -4.47099060e-02 -3.67634818e-...
section_10embedding,[-4.57181484e-02 -5.33069260e-02 -1.22106224e-...
section_4embedding,[-0.05072239 -0.01314292 -0.03440231 -0.023048...
section_8embedding,[-3.04003223e-02 4.79081871e-03 -5.41995295e-...
section_14embedding,[ 3.38502717e-03 -5.04764915e-03 -5.59636764e-...
section_1Aembedding,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...
section_11embedding,[-4.79705371e-02 -6.59401938e-02 -2.51594633e-...
section_15embedding,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...


In [46]:
df_embedding_only.iloc[0].info()

<class 'pandas.core.series.Series'>
Index: 20 entries, section_7embedding to section_1embedding
Series name: 0
Non-Null Count  Dtype                      
--------------  -----                      
20 non-null     list<item: double>[pyarrow]
dtypes: list<item: double>[pyarrow](1)
memory usage: 120.8+ KB


In [45]:
type(df_embedding_only.iloc[0])

pandas.core.series.Series

In [52]:
import numpy as np

In [51]:
def calculate_mean(row):
  vector_df = pd.DataFrame(row.to_list())
  averaged_embedding_series = vector_df.mean(axis = 0)
  return averaged_embedding_series.to_list()

# Apply the function row-wise to create a new column with the mean
df_embedding['overall_embedding'] = df_embedding_only.apply(calculate_mean, axis=1)

# Display the first few rows to verify the result
print(df_embedding.head())

           filename      cik  year  \
0    50104_1993.txt    50104  1993   
1   315189_1993.txt   315189  1993   
2     1800_1993.txt     1800  1993   
3  1137091_2020.htm  1137091  2020   
4  1067983_2020.htm  1067983  2020   

                                  section_7embedding  \
0  [-3.47243063e-02  1.94188505e-02 -8.92389864e-...   
1  [-1.35571547e-02 -1.56442206e-02 -2.92028040e-...   
2  [-0.01756296 -0.01733708 -0.01346771 -0.013578...   
3  [-2.10153785e-02  2.59747695e-03 -3.92209039e-...   
4  [-3.98591966e-04 -6.93545857e-03 -3.90583078e-...   

                                  section_9embedding  \
0  [-1.55314188e-02 -2.45964527e-03 -3.88711654e-...   
1  [-1.83812398e-02  1.81908242e-03 -2.49630474e-...   
2  [-1.56782791e-02 -6.42858818e-03 -3.48928086e-...   
3  [-4.25587595e-02 -2.09636036e-02 -6.31605536e-...   
4  [-3.78713012e-02 -2.25618221e-02 -6.61137030e-...   

                                 section_12embedding  \
0  [-3.79250757e-02 -4.47099060e-02 -3.67

In [53]:
def x(row):
  print(type(row))
df_embedding.to_pandas().apply(x, axis=1)


<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>


,0
0,None
1,None
2,None
3,None
4,None


In [54]:
df_embedding

,filename,cik,year,section_7embedding,section_9embedding,section_12embedding,section_10embedding,section_4embedding,section_8embedding,section_14embedding,...,section_13embedding,section_7Aembedding,section_1Bembedding,section_9Bembedding,section_2embedding,section_6embedding,section_3embedding,section_9Aembedding,section_1embedding,overall_embedding
0,50104_1993.txt,50104,1993,[-3.47243063e-02 1.94188505e-02 -8.92389864e-...,[-1.55314188e-02 -2.45964527e-03 -3.88711654e-...,[-3.79250757e-02 -4.47099060e-02 -3.67634818e-...,[-4.57181484e-02 -5.33069260e-02 -1.22106224e-...,[-0.05072239 -0.01314292 -0.03440231 -0.023048...,[-3.04003223e-02 4.79081871e-03 -5.41995295e-...,[ 3.38502717e-03 -5.04764915e-03 -5.59636764e-...,...,[-4.50329408e-02 -7.63122365e-02 -3.43960524e-...,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,[-1.83147509e-02 -4.80058193e-02 -3.25606801e-...,[ 1.19868275e-02 -3.50940004e-02 -3.42585109e-...,[-5.95068801e-02 2.10417886e-03 -2.92739635e-...,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,[-0.03292404 0.00087296 -0.04182 -0.004596...,[-0.02347898 -0.01731962 -0.02790547 0.000921...
1,315189_1993.txt,315189,1993,[-1.35571547e-02 -1.56442206e-02 -2.92028040e-...,[-1.83812398e-02 1.81908242e-03 -2.49630474e-...,[-3.85654159e-02 -1.78559888e-02 -4.98842299e-...,[-5.09253442e-02 -3.28539982e-02 -1.21700037e-...,[-4.47579399e-02 -1.16266981e-02 -4.29858454e-...,[-9.26065768e-05 -5.11534102e-02 -4.29588519e-...,[ 2.28945124e-02 5.21549085e-03 -1.74354800e-...,...,[-5.07865660e-02 -4.80615199e-02 -3.68813202e-...,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,[ 1.60061345e-02 2.85344739e-02 2.26290263e-...,[-1.48224207e-02 -4.50210385e-02 -6.59380779e-...,[-1.44546749e-02 -2.31604483e-02 -1.49410742e-...,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,[ 2.29252045e-02 2.62432406e-03 -4.36920724e-...,[-1.49562186e-02 -1.20997473e-02 -1.77254046e-...
2,1800_1993.txt,1800,1993,[-0.01756296 -0.01733708 -0.01346771 -0.013578...,[-1.56782791e-02 -6.42858818e-03 -3.48928086e-...,[-4.16508093e-02 -2.62772869e-02 -3.11164446e-...,[-6.22933321e-02 -4.06846032e-02 -2.88069844e-...,[-0.01235421 -0.05187324 -0.01839188 0.048709...,[-0.01758242 -0.04470409 -0.02377128 -0.002298...,[ 3.80702619e-03 -6.57393262e-02 -4.26834710e-...,...,[-4.13229093e-02 -6.52043819e-02 -4.08298895e-...,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,[-1.38139455e-02 -9.38619545e-04 -1.91317741e-...,[-2.47520432e-02 -3.38479057e-02 -2.15717610e-...,[-0.05118279 -0.03660664 -0.03556744 -0.023323...,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,[-8.46420446e-03 -1.74391275e-02 -3.04021587e-...,[-2.02630794e-02 -2.28220087e-02 -2.03435779e-...
3,1137091_2020.htm,1137091,2020,[-2.10153785e-02 2.59747695e-03 -3.92209039e-...,[-4.25587595e-02 -2.09636036e-02 -6.31605536e-...,[-6.73391223e-02 -1.49382157e-02 -3.27744223e-...,[-3.24487165e-02 -1.60200917e-03 -5.89513499e-...,[-3.31896767e-02 1.45743862e-02 -3.72849405e-...,[-1.28673098e-02 -8.76872710e-03 -2.86414517e-...,[-7.22742230e-02 -2.71525867e-02 -2.11981349e-...,...,[-0.05171408 -0.06189608 -0.03306692 0.002361...,[-4.44080308e-02 3.40547636e-02 -2.83501763e-...,[-6.44871220e-02 -3.82172340e-03 -1.57967471e-...,[-5.18303849e-02 -1.67017803e-02 -4.17982489e-...,[-9.54162516e-03 -8.53198580e-03 7.73510430e-...,[-0.02995299 -0.00425722 -0.03677021 0.041000...,[-5.38757481e-02 -4.11248840e-02 -2.54752636e-...,[-3.16515304e-02 -1.75493170e-02 -9.58232504e-...,[-5.22894418e-03 -6.85504824e-03 -1.68921131e-...,[-3.72444206e-02 -1.18594729e-02 -2.65914233e-...
4,1067983_2020.htm,1067983,2020,[-3.98591966e-04 -6.93545857e-03 -3.90583078e-...,[-3.78713012e-02 -2.25618221e-02 -6.61137030e-...,[0.

In [55]:
df_embedding.info()

<class 'bigframes.dataframe.DataFrame'>
Index: 5 entries, 0 to 4
Data columns (total 24 columns):
  #  Column               Non-Null Count    Dtype
---  -------------------  ----------------  ---------------------------
  0  filename             5 non-null        string
  1  cik                  5 non-null        Int64
  2  year                 5 non-null        Int64
  3  section_7embedding   5 non-null        list<item: double>[pyarrow]
  4  section_9embedding   5 non-null        list<item: double>[pyarrow]
  5  section_12embedding  5 non-null        list<item: double>[pyarrow]
  6  section_10embedding  5 non-null        list<item: double>[pyarrow]
  7  section_4embedding   5 non-null        list<item: double>[pyarrow]
  8  section_8embedding   5 non-null        list<item: double>[pyarrow]
  9  section_14embedding  5 non-null        list<item: double>[pyarrow]
 10  section_1Aembedding  5 non-null        list<item: double>[pyarrow]
 11  section_11embedding  5 non-null        list<item

In [56]:
df_embedding.to_pandas().iloc[:, 3:].applymap(len)

/tmp/ipython-input-2795087265.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_embedding.to_pandas().iloc[:, 3:].applymap(len)


,section_7embedding,section_9embedding,section_12embedding,section_10embedding,section_4embedding,section_8embedding,section_14embedding,section_1Aembedding,section_11embedding,section_15embedding,...,section_13embedding,section_7Aembedding,section_1Bembedding,section_9Bembedding,section_2embedding,section_6embedding,section_3embedding,section_9Aembedding,section_1embedding,overall_embedding
0,768,768,768,768,768,768,768,768,768,768,...,768,768,768,768,768,768,768,768,768,768
1,768,768,768,768,768,768,768,768,768,768,...,768,768,768,768,768,768,768,768,768,768
2,768,768,768,768,768,768,768,768,768,768,...,768,768,768,768,768,768,768,768,768,768
3,768,768,768,768,768,768,768,768,768,768,...,768,768,768,768,768,768,768,768,768,768
4,768,768,768,768,768,768,768,768,768,768,...,768,768,768,768,768,768,768,768,768,768


In [57]:
df_embedding.to_gbq("financial-fraud-detection-1.EDGAR.edgar_corpus_small_embeddings", if_exists='replace')

'financial-fraud-detection-1.EDGAR.edgar_corpus_small_embeddings'